## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [8]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.28372634e-01, 1.46920122e+00, 5.56258439e-01, 3.54784231e+00,
       1.35675709e+00, 2.77364372e-01, 3.77682273e+00, 3.04912313e+00,
       3.21109344e+00, 9.67517854e+00, 6.46322737e+00, 4.97575321e+00,
       4.06002838e+00, 9.10845528e+00, 1.37974871e+01, 1.39809809e+01,
       2.62346495e+00, 9.17127929e+00, 1.46373855e+00, 5.60169716e+00,
       2.78065147e+00, 2.19544694e+01, 8.60206995e+00, 1.45716717e+00,
       3.74766852e+00, 1.46204326e+01, 7.72216979e+00, 1.72411040e+01,
       9.85043481e-02, 6.95856307e+00, 1.74460716e+01, 2.52276551e+01,
       2.90738943e+01, 2.15363820e+01, 3.12368611e+01, 1.88176740e+01,
       2.23668849e+01, 8.06625826e+00, 2.51264385e+01, 1.00506187e+00,
       3.08112749e+01, 1.06441860e+01, 2.04449521e+01, 7.72362891e+00,
       4.24575117e+01, 2.23705319e+01, 1.01123868e+01, 3.10031780e+01,
       3.71733617e+01, 9.02783790e+00, 1.75083352e+01, 2.98396395e+01,
       7.94234922e-01, 4.76174904e+01, 7.07238245e+00, 4.98000719e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(5)

[0.5283726340595711,
 1.4692012150784355,
 0.5562584386212934,
 3.547842314794596,
 1.3567570911705458]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [16]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

In [17]:
revenue_minus_tax

PythonRDD[9] at RDD at PythonRDD.scala:53

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [18]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.4861028233348055,
 1.3516651178721608,
 0.51175776353159,
 3.2640149296110286,
 1.248216523876902]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x: x * 0.9)

In [20]:
discounted.take(10)

[0.43749254100132495,
 1.2164986060849448,
 0.46058198717843096,
 2.937613436649926,
 1.123394871489212,
 0.22965770015846645,
 3.1272092201645116,
 2.524673948748721,
 2.658785367655623,
 8.011047827477698]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [22]:
price_items.map(sales_tax).map(lambda x: x * 0.9).top(15)

[784.1012548609626,
 776.9799060443758,
 775.6303747880063,
 749.6419392657625,
 744.1785788992539,
 736.362593408591,
 728.3424086020233,
 725.0397489321634,
 719.1111554302489,
 714.4020481914838,
 713.1120667938706,
 712.9427748974813,
 703.8273704225867,
 698.4499499196039,
 693.2471144874952]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [23]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [25]:
mapped = price_items.map(lambda x: (x, x* 0.92 * 0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.5283726340595711, 0.43749254100132495), (1.4692012150784355, 1.2164986060849448), (0.5562584386212934, 0.46058198717843096), (3.547842314794596, 2.937613436649926), (1.3567570911705458, 1.123394871489212), (0.2773643721720609, 0.22965770015846645), (3.7768227296672845, 3.1272092201645116), (3.0491231265081167, 2.524673948748721), (3.211093439197612, 2.658785367655623), (9.675178535601086, 8.011047827477698)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [26]:
flat_mapped = price_items.flatMap(lambda x: (x, x* 0.92 * 0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.5283726340595711, 0.43749254100132495, 1.4692012150784355, 1.2164986060849448, 0.5562584386212934, 0.46058198717843096, 3.547842314794596, 2.937613436649926, 1.3567570911705458, 1.123394871489212]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [27]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

284

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [28]:
discounted.reduce(lambda a, b: a + b)

213688.92756957142

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [29]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(11, 300.43133314129545),
 (17, 312.14798810684334),
 (37, 317.16143896898456),
 (46, 313.87796205513365),
 (41, 318.55178724279926),
 (21, 338.2012679521263),
 (11, 349.2824752109123)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [33]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)

In [34]:
# sort the users from highest to lowest spenders
sorted_users = total_spent.sortBy(lambda x: x[1], ascending=False).collect()

In [35]:
sorted_users

[(34, 5562.088804088841),
 (9, 5209.390582258093),
 (22, 4573.8194619420065),
 (13, 4422.922935535855),
 (20, 4146.524323859714),
 (19, 3823.6975894163998),
 (25, 3772.2484226176744),
 (21, 3761.668898265527),
 (44, 3736.1270596630566),
 (7, 3708.307379549558),
 (35, 3618.28882091943),
 (48, 3454.308054744447),
 (12, 3350.6581451719057),
 (28, 3260.808122740852),
 (10, 3201.7327647377615),
 (41, 3194.881843592493),
 (4, 3110.267254563089),
 (24, 3084.70345443388),
 (42, 3036.2059423391943),
 (47, 2993.105022559959),
 (1, 2979.838360038198),
 (2, 2771.6521750583183),
 (29, 2764.217298169082),
 (49, 2757.6060759501634),
 (17, 2445.7741401443172),
 (38, 2440.911699095217),
 (50, 2409.2409422119244),
 (23, 2408.062752207603),
 (32, 2380.8261050971346),
 (45, 2226.6513982830006),
 (37, 2127.2858790330506),
 (33, 2014.446610721171),
 (11, 1867.8376585764436),
 (8, 1763.8577929409953),
 (14, 1736.8346363641335),
 (16, 1664.9269867056655),
 (15, 1615.9049858592357),
 (6, 1614.2883666232196),
 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [38]:
sales_data.countByKey()

defaultdict(int,
            {22: 8,
             34: 5,
             17: 4,
             14: 9,
             28: 6,
             31: 5,
             2: 5,
             36: 12,
             44: 11,
             37: 3,
             6: 3,
             8: 6,
             10: 6,
             18: 8,
             16: 7,
             41: 4,
             42: 5,
             43: 3,
             48: 6,
             45: 7,
             49: 4,
             38: 3,
             47: 2,
             11: 8,
             19: 3,
             40: 9,
             5: 8,
             13: 9,
             29: 6,
             32: 1,
             7: 3,
             4: 11,
             15: 9,
             25: 5,
             50: 3,
             26: 6,
             1: 3,
             3: 7,
             12: 5,
             30: 5,
             20: 3,
             24: 3,
             33: 3,
             21: 11,
             35: 5,
             9: 7,
             23: 2,
             27: 9,
             46: 4,
        

In [43]:
sorted(sales_data.countByKey().items(), key=lambda x: x[1], reverse=True)

[(33, 11),
 (45, 10),
 (25, 10),
 (6, 9),
 (38, 9),
 (43, 9),
 (34, 8),
 (48, 8),
 (46, 8),
 (13, 8),
 (9, 7),
 (8, 7),
 (42, 7),
 (49, 7),
 (39, 7),
 (24, 7),
 (16, 7),
 (4, 6),
 (22, 6),
 (2, 6),
 (50, 6),
 (40, 6),
 (37, 6),
 (21, 6),
 (14, 6),
 (44, 5),
 (1, 5),
 (19, 5),
 (47, 5),
 (7, 5),
 (29, 5),
 (20, 5),
 (5, 4),
 (17, 4),
 (18, 4),
 (31, 4),
 (35, 4),
 (12, 4),
 (11, 4),
 (10, 4),
 (30, 4),
 (36, 4),
 (3, 3),
 (27, 3),
 (41, 3),
 (23, 3),
 (26, 3),
 (15, 3),
 (28, 2),
 (32, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
